HEART FAILURE:
Fun day-long project with the express purpose of brushing up on techniques/brushing off the rust
This is the first installment and meant to be see how much I can get done in a day. Hope you enjoy!
- Parker

 Data taken from Kaggle "https://www.kaggle.com/andrewmvd/heart-failure-clinical-data "

In [ ]:
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
import pandas as pd

In [ ]:
# most of the cool toy datasets don't have much data...probably for the best


In [ ]:
Heart_data_path = "heart_failure_clinical_records_dataset.csv"
#Heart is way cooler to type than data everytime
Heart = pd.read_csv(Heart_data_path)

In [ ]:
#Lets open the patient up and see the problem
Heart.head()

In [ ]:
# Looks like we have a few binary inputs as medical conditions
# woman is a 0 and man is a 1.
# otherwise 0 is a no and 1 is a yes
# ~300 rows not a terribly large dataset...
# presumably we want sympotoms to predict death, and not vice versa

In [ ]:
#Set variables to predictors X and binary output Y
y = Heart.iloc[:,Heart.columns =="DEATH_EVENT"]
X = Heart.iloc[:,Heart.columns !="DEATH_EVENT"]


In [ ]:
#Random Forests are pretty clutch most of the time, let's try that out first
#(dataset pretty small too hmm that might be a problem)
RF = RandomForestClassifier(n_estimators=30, max_depth=9, random_state=0)
y = np.ravel(y)
RF.fit(X,y)
RF.predict(X.iloc[100:,:])
round(RF.score(X,y), 2)

In [ ]:
#As expected, if we train on the entire dataset, and have a large depth we're going to get a perfect score.....
# which is useless.... and I should have remembered this....
#lets plot this effect anyway

score_plot = [0 for x in range(11)]
for i in range(11):
    RF = RandomForestClassifier(n_estimators=30, max_depth=i+1, random_state=0)
    y = np.ravel(y)
    RF.fit(X,y)
    RF.predict(X.iloc[100:,:])
    score_plot[i] = round(RF.score(X,y), 2)

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
#Define depth as X- axis and score on Y
plt.plot(range(1,12) , score_plot)

In [ ]:
# depth seems pretty important here, but it's definitely prone to overfitting without validation
#but cleary this random forest is also a function of estimators,
#curious to know what an optimal estimator with minimal depth combination is
# lets nest a couple loops and see what happens

In [ ]:
#Warning: this could take a few seconds, we're running the RF 250 times with varying computational budgets
score_plot2D = [[np.nan for x in range(25)] for y in range(11)]
for j in range(2,25):
    for i in range(11):
        RF = RandomForestClassifier(n_estimators=j+1, max_depth=i+1, random_state=0)
        y = np.ravel(y)
        RF.fit(X,y)
        RF.predict(X.iloc[100:,:])
        score_plot2D[i][j] = round(RF.score(X,y), 2)


In [ ]:
plt.imshow(score_plot2D, aspect="auto", interpolation=None, vmin=0.7, vmax=1)
plt.show()

In [ ]:
# number of estimators is slightly underwhelming in this dataset. Definitely keeping it above 5 though...


from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification


# time to do some proper data science with a real validation set

In [ ]:
#Reset this RF classifier parameters from previous test
RF = RandomForestClassifier(n_estimators=25, max_depth=5, random_state=0)

# introduce an actual training and validation set
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

#Same perscription as before now
RF.fit(train_X,train_y)
from sklearn.model_selection import cross_val_score
scores = round(cross_val_score(RF, val_X, val_y, cv=5).mean(), 2)
scores.mean()

In [ ]:
#I want to see the effect of depth now when a validation set is introduced..
score_plot_validated = [0 for x in range(11)]
for i in range(11):
    RF = RandomForestClassifier(n_estimators=30, max_depth=i+1, random_state=0)
    RF.fit(train_X,train_y)
    score_plot_validated[i] = cross_val_score(RF,val_X,val_y).mean()

In [ ]:
plt.plot(range(1,12) , score_plot_validated)

The above looks much less egregious with respect to model depth, and we have a model with an 89% correctness rate on our validation set (I'm pretty sure that's the correct interpretation in a classification model score)

Let's push the model by increasing the estimators by an order of magnitude to get those juicy dimishing returns on performance

In [ ]:
RF = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=0)
y = np.ravel(y)
RF.fit(train_X,train_y)
round(cross_val_score(RF,val_X,val_y).mean(),2)

Looks like the statistical variance was greater than any return I'd get by increasing the number of estimators..
Let's find some cool ways to score this and maybe adjust some features

In [ ]:
from sklearn.metrics import confusion_matrix
predict_y= RF.predict(val_X)
confusion_matrix(val_y, predict_y, normalize = "all")


 With matrix containing
 
                   Death			no-death
                   
    Death		true positive		false positive

    no-death	false negative		true negative
    
It looks like the false-positive and false-negative rates are simillar to within 60%


In [ ]:
#Try to find redundant features with respect to predicting mortality?
import seaborn
plt.figure(figsize=(12,10))
cor = Heart.corr()
seaborn.heatmap(cor, annot=True, cmap=plt.cm.Blues)
plt.show()


Nothing here stands out as a large correlation within the features. Some features correlate with Death events more weakly than others.

Let's see what happens when we drop the weak correlations with death-events themselves (gee I hope the definition of correlation is normalized...)

In [ ]:
Heart = Heart.drop(['sex', 'smoking'], axis = 1)
Heart.head()

In [ ]:
#one more time without those columns!
y = Heart.iloc[:,Heart.columns =="DEATH_EVENT"]
X = Heart.iloc[:,Heart.columns !="DEATH_EVENT"]
y = np.ravel(y)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
RF = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=0)
train_y = np.ravel(train_y)
RF.fit(train_X,train_y)
round(cross_val_score(RF,val_X,val_y).mean(),2)

Those two columns clearly have some decision effects somewhere. So I learned not to ignore free data....
But in the end we have a model that works pretty well, and with time constraints I am pretty happy with the insights gained